<a href="https://colab.research.google.com/github/SDog6/FruitPunchAIClassificationProject/blob/main/CoralReefsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pylabel > /dev/null

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns 
from matplotlib import pyplot as plt
from google.colab import drive
import cv2
from tqdm import tqdm
import os
import glob
from PIL import Image
from numpy import asarray
from pylabel import importer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Specify path to the coco.json file
path_to_annotations = "/content/drive/MyDrive/annotations/instances_train.json"
#Specify the path to the images (if they are in a different folder than the annotations)
path_to_images = "/content/drive/MyDrive/images/train"

#Import the dataset into the pylable schema 
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="BCCD_coco")
dataset.df


,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/content/drive/MyDrive/images/train,video53_1_000123.PNG,,124,640,470,,,317.53,1.16,...,[],0,,,,1,Human,,,1
1,/content/drive/MyDrive/images/train,video53_1_000124.PNG,,125,640,470,,,318.30,2.03,...,[],0,,,,1,Human,,,1
2,/content/drive/MyDrive/images/train,video53_1_000125.PNG,,126,640,470,,,319.08,2.90,...,[],0,,,,1,Human,,,1
3,/content/drive/MyDrive/images/train,video53_1_000126.PNG,,127,640,470,,,319.85,3.77,...,[],0,,,,1,Human,,,1
4,/content/drive/MyDrive/images/train,video53_1_000127.PNG,,128,640,470,,,320.08,4.24,...,[],0,,,,1,Human,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34193,/content/drive/MyDrive/images/train,video78_1_001153.PNG,,29687,640,470,,,282.53,458.41,...,[],0,,,,1,Human,,,1
34194,/content/drive/MyDrive/images/train,video78_1_001154.PNG,,29688,640,470,,,282.82,460.10,...,[],0,,,,1,Human,,,1
34195,/content/drive/MyDrive/images/train,video78_1_001155.PNG,,29689,640,470,,,283.32,461.90,...,[],0,,,,1,Human,,,1


In [ ]:
dataset.path_to_annotations = "data/yolo"

In [ ]:
dataset.export.ExportToYoloV5()

In [ ]:
dataset.path_to_annotations = "data/XML"

In [ ]:
dataset.export.ExportToVoc()

In [ ]:
dataset.df

In [ ]:

for x in os.listdir(r"/content/data/XML"):
  df = pd.read_xml("/content/data/XML" + x)
df

In [ ]:
from xml.etree import ElementTree
#In this variable we're going to make a bunch of empty that we're going to fill with the annotation data.
information = {'File': [],'AnnotationFile': [],'width': [], 'height': [], 'xmin': [], 'xmax': [], 'ymin': [], 'ymax': [], 'name': []}

#We're going to use the default glob module that comes pre-install with phyton to import the annotation file.
#With the ElementTree module we're going to go through and read each anotation file.
annotationDir = "/content/data/XML"
for annotation in tqdm(glob.glob(annotationDir + '/*.xml')):
    tree = ElementTree.parse(annotation)
    
    #Every XML file is made out of elements, and our XML files are made out of: 1 size element and 1 or more object element(s).
    for element in tree.iter():
        #The size element will tell us about the width and height of the picture.
        if 'size' in element.tag:
            #Here we loop through to each attribute in the element and we take only the width and the height.
            for attribute in list(element):
                if 'width' in attribute.tag: 
                    width = round(float(attribute.text))
                if 'height' in attribute.tag:
                    height = round(float(attribute.text))

        #The object element(s) represent the labeled face(s) in the specific image that is connected to the xml file.           
        if 'object' in element.tag:
            for attribute in list(element):
                #The name atribute is the actual lable of a specific face, telling us whether the face is wearing,
                #not wearing or weariing incorrectly a mask.
                if 'name' in attribute.tag:
                    name = attribute.text 
                    #The reason I did this line of code because when reading some images,the slashes in the path sometimes
                    #incorrectly get revesed e.g. ('/data/input/face-mask-detection' becomes /data\input/face-mask-detection).
                    newPath = annotation.replace(os.sep, '/')
                    #Here we will add the information that we have such as label, width, height, and the file name to the arrays.
                    information['name'] += [name]
                    information['width'] += [width]
                    information['height'] += [height]
                    information['File'] += [newPath.split('/')[-1][0:-4]]  
                    information['AnnotationFile'] += [annotation.split('/')[-1][0:-4]]


                
                #The bndbox (bounding box) attribute contains 2 longitudes and 2 latitudes that draw an outline around the face.
                if 'bndbox' in attribute.tag:
                    for dimension in list(attribute):
                        if 'xmin' in dimension.tag:
                            xmin = round(float(dimension.text))
                            information['xmin'] += [xmin]
                        if 'ymin' in dimension.tag:
                            ymin = round(float(dimension.text))
                            information['ymin'] += [ymin]                               
                        if 'xmax' in dimension.tag:
                            xmax = round(float(dimension.text))
                            information['xmax'] += [xmax]                                
                        if 'ymax' in dimension.tag:
                            ymax = round(float(dimension.text))
                            information['ymax'] += [ymax]

In [ ]:
annotations_info_df = pd.DataFrame(information)
annotations_info_df

In [ ]:
annotations_info_df['AnnotationFile'] = annotations_info_df['AnnotationFile'] + '.xml'
annotations_info_df['ImageFile'] = annotations_info_df['File'] + '.PNG'

annotations_info_df

In [214]:
path = "/content/data/CroppeddImage"
if os.path.isdir(path) is False:
    os.mkdir(path)

In [ ]:
OutputFolder = "/content/data/OutputImages"

In [ ]:
annotations_info_df['cropped_image_file'] = annotations_info_df['File']
annotations_info_df

In [ ]:
ImagesRemovedErrors = annotations_info_df[(annotations_info_df["xmin"]!=0) & (annotations_info_df["xmax"]!=0)]
ImagesRemovedErrors = ImagesRemovedErrors[(ImagesRemovedErrors["ymin"]!=0) & (ImagesRemovedErrors["ymax"]!=0)]
ImagesRemovedErrors

In [ ]:
for i in tqdm(range(len(ImagesRemovedErrors))):
    image_filepath = '/content/data/OutputImages/' + ImagesRemovedErrors['ImageFile'].iloc[i]
    image = cv2.imread(image_filepath)    
    #In here we use the cropped_image_file column to set the cropped image file name.
    ImagesRemovedErrors['cropped_image_file'].iloc[i] = ImagesRemovedErrors['cropped_image_file'].iloc[i] + '-' + str(i) + '.png'
    cropped_image_filename = ImagesRemovedErrors['cropped_image_file'].iloc[i]
    
    #In here we're get the xmin, ymin, xmax, ymax value from the table to crop the face(s).
    xmin = ImagesRemovedErrors['xmin'].iloc[i]
    ymin = ImagesRemovedErrors['ymin'].iloc[i]
    xmax = ImagesRemovedErrors['xmax'].iloc[i]
    ymax = ImagesRemovedErrors['ymax'].iloc[i]

    #In here we're actually cropping the face(s) based on the values from the xmin, ymin, xmax, ymax.
    cropped__img = image[ymin:ymax, xmin:xmax]
    #After the the face(s) completely cropped we're going to save the face(s) on the directory that we already made.
    cropped_image_directory = os.path.join('/content/data/CroppeddImage', cropped_image_filename)
    cv2.imwrite(cropped_image_directory, cropped__img)

In [212]:
len(os.listdir("/content/data/CroppeddImage"))

33786

In [215]:
path = "/content/data/CroppeddImage"
newPath = "/content/data/ResizeImage"
#Here we make a new directory if the directory is not exist.
if os.path.isdir(newPath) is False:
    os.mkdir(newPath)
#In here I define my directory.
dirs = os.listdir( path )
#Here is the method to resize the images.
def resize():
    #Here we loop through the images.
    for item in dirs:
        #If the file path is right we go through the image file.
        if os.path.isfile(path+item):
            #Here we go into the image file.
            image = Image.open(path+item)
            #Here I resize the image into 200x200, and  the function of ANTIALIAS is to prevent the image from distorting.
            imResize = image.resize((200,200), Image.ANTIALIAS)
            #Here I save the image, I put the quality in 90 so we get the best quality of the images.
            imResize.save( newPath + item + ' resized', 'PNG', quality=90)
resize()